In [1]:
import numpy as np
np.set_printoptions(precision=4, suppress=True)

import equinox as eqx
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import importlib
import numpy as np
import copy

from jax import grad, jit, vmap
from jax.experimental.ode import odeint
from IPython.display import HTML
from time import time

from typing import TYPE_CHECKING, Callable, Union, Optional

from VariablesClass import VariablesClass
from StructureClass import  StructureClass
from StateClass import StateClass
from EquilibriumClass import EquilibriumClass
from config import CFG

import plot_funcs, colors, helpers_builders, learning_funcs, file_funcs, numerical_experiments

## Predetermined trajectory

In [ ]:
## load

importlib.reload(file_funcs)
importlib.reload(helpers_builders)

_, _, F_exp_old_11100 = file_funcs.load_pos_force("C:\\Users\\roiee\OneDrive - huji.ac.il\\PhD\Research\\Robotic arm\\Meca500\\data\\measurements\\Jan20\\old chain\\buckle_00011.csv", mod="arrays")
_, _, F_exp_new_11100 = file_funcs.load_pos_force("C:\\Users\\roiee\OneDrive - huji.ac.il\\PhD\Research\\Robotic arm\\Meca500\\data\\measurements\\Jan20\\new chain\\buckle_00011.csv", mod="arrays")
_, _, F_exp_old_11111 = file_funcs.load_pos_force("C:\\Users\\roiee\OneDrive - huji.ac.il\\PhD\Research\\Robotic arm\\Meca500\\data\\measurements\\Jan20\\old chain\\buckle_00000.csv", mod="arrays")
_, _, F_exp_new_11111 = file_funcs.load_pos_force("C:\\Users\\roiee\OneDrive - huji.ac.il\\PhD\Research\\Robotic arm\\Meca500\\data\\measurements\\Jan20\\new chain\\buckle_00000.csv", mod="arrays")
_, P, F_sim_11100 = file_funcs.load_pos_force("C:\\Users\\roiee\\OneDrive - huji.ac.il\\PhD\Research\\Bistable shape acquisition jax\\comparison_sim_experment\\Jan20\\L=0.045_buckle[-1 -1 -1  1  1].csv", mod="arrays")
_, _, F_sim_11111 = file_funcs.load_pos_force("C:\\Users\\roiee\\OneDrive - huji.ac.il\\PhD\Research\\Bistable shape acquisition jax\\comparison_sim_experment\\Jan20\\L=0.045_buckle[-1 -1 -1 -1 -1].csv", mod="arrays")
y = P[:, 1]

In [ ]:
# specific buckle arangement, experiment and sim

# solid lines (1st and 2nd)
plt.plot(y * 1000, F_exp_old_11100[:, 0], color=colors_lst[0], marker='.', linestyle='None')
plt.plot(y * 1000, F_exp_new_11100[:, 0], color=colors_lst[0], marker='o', linestyle='None')
plt.plot(y * 1000, F_sim_11100[:, 0]/1000, color=colors_lst[0], linestyle='-')
plt.plot(y * 1000, F_exp_old_11100[:, 1], color=colors_lst[1], marker='.', linestyle='None')
plt.plot(y * 1000, F_exp_new_11100[:, 1], color=colors_lst[1], marker='o', linestyle='None')
plt.plot(y * 1000, F_sim_11100[:, 1]/1000, color=colors_lst[1], linestyle='-')

# # dotted lines (3rd and 4th)
# plt.plot(P[:, 1] * 1000, F_x_vec / 1000, color=colors_lst[0], linestyle=':')
# plt.plot(P[:, 1] * 1000, F_y_vec / 1000, color=colors_lst[1], linestyle=':')

plt.ylabel('F [N]')
plt.xlabel('y [mm]')
plt.legend(['Fx (exp old)', 'Fx (exp new)', 'Fx (model)', 'Fy (exp old)', 'Fy (exp new)', 'Fy (model)'])

In [ ]:
# another buckle arangement, experiment and sim

# solid lines (1st and 2nd)
plt.plot(y * 1000, F_exp_old_11111[:, 0], color=colors_lst[0], marker='.', linestyle='None')
plt.plot(y * 1000, F_exp_new_11111[:, 0], color=colors_lst[0], marker='o', linestyle='None')
plt.plot(y * 1000, F_sim_11111[:, 0]/1000, color=colors_lst[0], linestyle='-')
plt.plot(y * 1000, F_exp_old_11111[:, 1], color=colors_lst[1], marker='.', linestyle='None')
plt.plot(y * 1000, F_exp_new_11111[:, 1], color=colors_lst[1], marker='o', linestyle='None')
plt.plot(y * 1000, F_sim_11111[:, 1]/1000, color=colors_lst[1], linestyle='-')

# # dotted lines (3rd and 4th)
# plt.plot(P[:, 1] * 1000, F_x_vec / 1000, color=colors_lst[0], linestyle=':')
# plt.plot(P[:, 1] * 1000, F_y_vec / 1000, color=colors_lst[1], linestyle=':')

plt.ylabel('F [N]')
plt.xlabel('y [mm]')
plt.legend(['Fx (exp old)', 'Fx (exp new)', 'Fx (model)', 'Fy (exp old)', 'Fy (exp new)', 'Fy (model)'])

## ADMET stress strain

In [ ]:
## --- import CSVs and plot ---

importlib.reload(file_funcs)
importlib.reload(plot_funcs)

# exp_path = "Experimental ADMET Bi-ax\\Oct4\\leave_every_time\\Roee_buckle11111_bellyDown.csv"
# exp_path = "Experimental ADMET Bi-ax\\Oct3\\Roee_buckle11111_init190mm_inside150mm_bellyDown.csv"
# exp_path = "Experimental ADMET Bi-ax\\Oct5\\BellyUp_2ndshims_1stattempt.csv"

## buckle [1 1 1 1 1] Oct 5

## belly down
# exp_path_1 = "Experimental ADMET Bi-ax\\Oct5\\bellyDown_1stshims_1stattempt.csv"
# exp_path_2 = "Experimental ADMET Bi-ax\\Oct5\\bellyDown_2ndshims_1stattempt.csv"
# exp_path_3 = "Experimental ADMET Bi-ax\\Oct5\\bellyDown_1stshims_2ndattempt.csv"
# exp_df_1 = file_funcs.import_stress_strain_exp_and_plot(exp_path_1, plot=False)
# exp_df_2 = file_funcs.import_stress_strain_exp_and_plot(exp_path_2, plot=False)
# exp_df_3 = file_funcs.import_stress_strain_exp_and_plot(exp_path_3, plot=False)
# exp_dfs = [exp_df_1, exp_df_2, exp_df_3]
# sim_path = "L=1_buckle[1 1 1 1 1] belly down.csv"
# sim_df = file_funcs.import_stress_strain_sim_and_plot(sim_path, plot=False)

## belly up
exp_path_1 =  "Experimental ADMET Bi-ax\\Oct5\\BellyUp_1stshims.csv"
exp_path_2 =  "Experimental ADMET Bi-ax\\Oct5\\BellyUp_2ndshims_1stattempt.csv"
exp_df_1 = file_funcs.import_stress_strain_exp_and_plot(exp_path_1, plot=False)
exp_df_2 = file_funcs.import_stress_strain_exp_and_plot(exp_path_2, plot=False)
exp_dfs = [exp_df_1, exp_df_2]
# sim_path = "Simulation stress strain\\Nov17\\L=1_buckle[1 1 1 1 1] belly up.csv"
sim_path = "L=1_buckle[1 1 1 1 1] belly up.csv"
sim_df = file_funcs.import_stress_strain_sim_and_plot(sim_path, plot=False)

## buckle [1 0 0 0 1] Oct 4
# exp_path_1 = "Experimental ADMET Bi-ax\\Oct4\\dismantle_every_time\\Roee_buckle10001_bellyUp_level_1st.csv"
# exp_path_2 = "Experimental ADMET Bi-ax\\Oct4\\dismantle_every_time\\Roee_buckle10001_bellyUp_level_2nd.csv"
# exp_path_3 = "Experimental ADMET Bi-ax\\Oct4\\dismantle_every_time\\Roee_buckle10001_bellyUp_level_3rd.csv"
# exp_df_1 = file_funcs.import_stress_strain_exp_and_plot(exp_path_1, plot=False)
# exp_df_2 = file_funcs.import_stress_strain_exp_and_plot(exp_path_2, plot=False)
# exp_df_3 = file_funcs.import_stress_strain_exp_and_plot(exp_path_3, plot=False)
# exp_dfs = [exp_df_1, exp_df_2, exp_df_3]
# sim_path = "Simulation stress strain\\Nov17\\L=1_buckle[1 0 0 0 1] belly up.csv"
# sim_df = file_funcs.import_stress_strain_sim_and_plot(sim_path, plot=False)


plot_funcs.plot_compare_sim_exp_stress_strain(exp_dfs, sim_df, 1000)